In [13]:
import pandas as pd
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForSequenceClassification
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
# from utils import pad_sents
import sys

In [4]:
df = pd.read_csv('preprocessed_data.csv')

In [6]:
df.head()

,Unnamed: 0,text,target,ProcessedText_BERT,ProcessedText_BERTbase_length,ProcessedText_BERTlarge_length,InformationType_label
0,0,our deeds are the reason of this # earthquake ...,1,[CLS] our deeds are the reason of this # earth...,15,15,5759
1,1,forest fire near la ronge sask . canada,1,[CLS] forest fire near la ronge sask . canada,11,11,801
2,2,all residents asked to ' shelter in place ' ar...,1,[CLS] all residents asked to ' shelter in plac...,26,26,5781
3,3,"13 , 000 people receive # wildfires evacuation...",1,"[CLS] 13 , 000 people receive # wildfires evac...",13,13,3882
4,4,just got sent this photo from ruby # alaska as...,1,[CLS] just got sent this photo from ruby # ala...,21,21,5374


In [10]:
df['ProcessedText_BERT']

0       [CLS] our deeds are the reason of this # earth...
1           [CLS] forest fire near la ronge sask . canada
2       [CLS] all residents asked to ' shelter in plac...
3       [CLS] 13 , 000 people receive # wildfires evac...
4       [CLS] just got sent this photo from ruby # ala...
5       [CLS] # rockyfire update => california hwy . 2...
6       [CLS] # flood # disaster heavy rain causes fla...
7       [CLS] i ' m on top of the hill and i can see a...
8       [CLS] there ' s an emergency evacuation happen...
9       [CLS] i ' m afraid that the tornado is coming ...
10      [CLS] three people died from the heat wave so far
11      [CLS] haha south tampa is getting flooded hah ...
12      [CLS] # raining # flooding # florida # tampaba...
13        [CLS] # flood in bago myanmar # we arrived bago
14      [CLS] damage to school bus on 80 in multi car ...
15                                [CLS] what ' s up man ?
16                                [CLS] my car is so fast
17            

### Sentences to Tensor

In [47]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForSequenceClassification
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
# from utils import pad_sents
import sys

In [34]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [35]:
tokens_list = [tokenizer.tokenize(sent) for sent in list(df['ProcessedText_BERT'])]

In [37]:
sents_lengths = [len(tokens) for tokens in tokens_list]

In [45]:
def pad_sents(sents, pad_token):
    """ Pad list of sentences according to the longest sentence in the batch.
    @param sents (list[list[int]]): list of sentences, where each sentence
                                    is represented as a list of words
    @param pad_token (int): padding token
    @returns sents_padded (list[list[int]]): list of sentences where sentences shorter
        than the max length sentence are padded out with the pad_token, such that
        each sentences in the batch now has equal length.
        Output shape: (batch_size, max_sentence_length)
    """
    sents_padded = []

    max_len = max(len(s) for s in sents)
    batch_size = len(sents)

    for s in sents:
        padded = [pad_token] * max_len
        padded[:len(s)] = s
        sents_padded.append(padded)

    return sents_padded

In [46]:
tokens_list_padded = pad_sents(tokens_list, '[PAD]')

In [48]:
sents_lengths = torch.tensor(sents_lengths)

In [49]:
masks = []
for tokens in tokens_list_padded:
    mask = [0 if token=='[PAD]' else 1 for token in tokens]
    masks.append(mask)
    
masks_tensor = torch.tensor(masks, dtype=torch.long)
tokens_id_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokens_list_padded]
sents_tensor = torch.tensor(tokens_id_list, dtype=torch.long)

### Default Model

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
pre_softmax = bert(input_ids= sents_tensor, attention_mask= masks_tensor)

# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# outputs = model(input_ids, labels=labels)
# loss, logits = outputs[:2]

100%|██████████| 407873900/407873900 [06:18<00:00, 1077568.71B/s]
